# 네이버 영화리뷰 감정 분석 문제에 SentencePiece 적용해 보기

네이버 영화리뷰 감정 분석 태스크가 있습니다. 한국어로 된 corpus를 다루어야 하므로 주로 KoNLPy에서 제공하는 형태소 분석기를 사용하여 텍스트를 전처리해서 RNN 모델을 분류기로 사용하게 되는데요.

만약 이 문제에서 tokenizer를 SentencePiece로 바꾸어 다시 풀어본다면 더 성능이 좋아질까요? KoNLPy에 있는 Mecab, kkma, Okt 등과 비교해보세요. (여러분들은 fasttext로 사전훈련된 Word Vector를 사용할 수 있지만 sentencepiece와 KoNLPy에 있는 형태소로 모델을 만드는 것보다 코드 수정이 많이 일어납니다. 기본적인 태스크를 끝나고(sentencepiece - KoNLPy 형태소 비교) 도전하시는걸 추천합니다.)

## 프로젝트: SentencePiece 사용하기

In [12]:
import tensorflow as tf
import numpy as np
import matplotlib as plt
import konlpy

print(tf.__version__)
print(np.__version__)
print(plt.__version__)
print(konlpy.__version__)

In [13]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter
from sklearn.model_selection import train_test_split

from konlpy.tag import Hannanum,Kkma,Komoran,Mecab,Okt

In [14]:
from tqdm import tqdm, trange

tqdm.pandas()

In [15]:
import os, re

## 데이터 불러오기

In [16]:
train_path = './data/ratings_train.txt'
test_path = './data/ratings_test.txt'


train = pd.read_table(train_path)
test = pd.read_table(test_path)

In [17]:
train.head()

In [18]:
test.head()

In [19]:
print(f"train shape => {train.shape} \ntest shape => {test.shape}")

In [20]:
train.columns

## 훈련 데이터 라벨링 값 비율 확인

In [21]:
sns.set_theme(style="darkgrid")
ax = sns.countplot(x="label", data=train)

In [22]:
labels, frequencies = np.unique(train.label.values, return_counts=True)
plt.figure(figsize=(5,5))
plt.pie(frequencies, labels = labels, autopct= '%1.1f%%')
plt.show()

## 훈련, 테스트 데이터 결측치 값 확인

In [23]:
train.isnull().sum()

In [24]:
test.isnull().sum()

In [25]:
train.drop_duplicates(subset=['document'], inplace=True)
test.drop_duplicates(subset=['document'], inplace=True)
train.dropna(inplace=True)
test.dropna(inplace=True)

# 전처리
## 1) train, test data의 문장 길이 확인

In [26]:
train_len = train.document.apply(lambda x: len(x))

In [27]:
train_len.describe()

In [28]:
train_len.hist()

In [29]:
test.document.apply(lambda x: len(x)).hist()

- train len과 test len의 분포를 비교해본 결과 상당히 유사해서 길이를 줄이는 등의 전처리는 진행하지 않는 것이 좋을 것 같았다

## 2) 길이가 너무 길거나 짧은 실제 데이터 확인 

- 한글자의 경우도 의미가 없지 않을 까 확인해봤는데 굿, 욜 이런 부분에서 의미가 있는 부분도 있었다
- 문자를 다 삭제하는 것도 고민해봐야겠다. ♥,♡,乃, ㅄ 도 의미가 있어보인다

In [30]:
train.loc[train_len[train_len > 145].index].document

In [31]:
' '.join(train.loc[train_len[train_len == 1].index].document)

- 한글자의 경우도 의미가 없지 않을 까 확인해봤는데 굿, 욜 이런 부분에서 의미가 있는 부분도 있었다
- 문자를 다 삭제하는 것도 고민해봐야겠다. ♥,♡,乃, ㅄ 도 의미가 있어보인다

In [32]:
import re

### 반복되는 문자 처리 추가

In [33]:
i = '숨은글씨 찾기 의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리니의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리'

re.sub('(\\S{2})\\1+', '\\1', i)

In [34]:
len(train)

In [ ]:
### 최종 전처리 함수
- 한글, 숫자, 초성, ♥♡乃ㄳㅄ의 문자를 남겼다
- 초성 ㅜㅜ, ㅋㅋ이 감정의 의미를 담을 수 있다고 파악했기 때문이다
- 다중 공백을 제거하는 코드를 추가하였다
- ㅋ와 ㅋㅋ은 다르게 인지하였다
    - ㅜㅜㅜ, ㅋㅋㅋㅋㅋㅋ 등의 다중 초성은 2개의 초성으로 모두 합쳐주었다
- 빈 하트와 채워진 하트는 채워진 하트로 통일하고 하트의 갯수는 모두 하나로 바꿔주었다
- 반복되는 문자열은 하나만 남기고 지워줬다

In [36]:
def preprocessing(train, col='document'):
    train[col] = train[col].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣0-9♥♡乃ㄳㅄ ]","") # 정규 표현식 수행
    train[col] = train[col].str.replace('^ +', "") # 공백은 empty 값으로 변경
    train[col].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
    train.dropna(how='any', inplace=True)
    
    train[col] = train[col].apply(lambda x: ' '.join(x.split()))
    train[col] = train[col].apply(lambda x: re.sub(r'(ㅋ|ㅎ|ㅜ|ㅠ){2,}', lambda m: m.group(1) * 2, x)) # 반복되는 2자리 문자 처리

    train[col] = train[col].str.replace(r'♡','♥')
    train[col] = train[col].apply(lambda x: re.sub(r'♥+', '♥', x))
    train[col] = train[col].apply(lambda x: re.sub(r'\b(\S+)( \1)+', r'\1', x))
    
    return train

In [37]:
train = preprocessing(train, col='document')

In [38]:
train.document[train.document.str.contains('♥|♡')].tolist()[:10]

In [39]:
train.document.apply(lambda x: len(x)).hist()

In [40]:
train.document.apply(lambda x: len(x)).describe()

In [41]:
train_len = train.document.apply(lambda x: len(x))

train.loc[train_len[train_len > 135].index].document.tolist()[:10]

In [42]:
corpus = train.document.tolist()

### 네이버 영화리뷰 감정 분석 코퍼스에 SentencePiece를 적용시킨 모델 학습하기

In [32]:
import sentencepiece as spm
import os

temp_file = os.getenv('HOME')+'/aiffel/sp_tokenizer/data/korean-english-nsmc.train.ko.temp'

vocab_size = 8000

with open(temp_file, 'w') as f:
    for row in corpus:   # 이전에 나왔던 정제했던 corpus를 활용해서 진행해야 합니다.
        f.write(str(row) + '\n')

spm.SentencePieceTrainer.Train(
    '--input={} --model_prefix=korean_spm --vocab_size={}'.format(temp_file, vocab_size)    
)

In [33]:
#위 Train에서  --model_type = unigram이 디폴트 적용되어 있습니다. --model_type = bpe로 옵션을 주어 변경할 수 있습니다.

!ls -l korean_spm*

In [55]:
sample = '아기자기하고 순수한이런영화좋다♥해피엔딩 결말도굿♥우울했는데 영화보고 힐링♥'

In [56]:
s = spm.SentencePieceProcessor()
s.Load('korean_spm.model')

# SentencePiece를 활용한 sentence -> encoding
tokensIDs = s.EncodeAsIds(sample)
print(tokensIDs)

# SentencePiece를 활용한 sentence -> encoded pieces
print(s.SampleEncodeAsPieces(sample,1, 0.0))

# SentencePiece를 활용한 encoding -> sentence 복원
print(s.DecodeIds(tokensIDs))

## Tokenizer 함수 작성

우리는 위에서 훈련시킨 SentencePiece를 활용하여 위 함수와 유사한 기능을 하는 sp_tokenize() 함수를 정의할 겁니다. 하지만 SentencePiece가 동작하는 방식이 단순 토큰화와는 달라 완전히 동일하게는 정의하기 어렵습니다. 그러니 아래 조건을 만족하는 함수를 정의하도록 하습니다.

1. 매개변수로 토큰화된 문장의 list를 전달하는 대신 온전한 문장의 list 를 전달합니다.

1. 생성된 vocab 파일을 읽어와 { <word> : <idx> } 형태를 가지는 word_index 사전과 { <idx> : <word>} 형태를 가지는 index_word 사전을 생성하고 함께 반환합니다.

1. 리턴값인 tensor 는 앞의 함수와 동일하게 토큰화한 후 Encoding된 문장입니다. 바로 학습에 사용할 수 있게 Padding은 당연히 해야겠죠?

### 학습된 모델로 sp_tokenize() 메소드 구현하기

In [39]:
test = preprocessing(test)

In [40]:
test_data = test.document.tolist()

In [41]:
import re

In [42]:
def sp_tokenize(s, corpus): 

    tensor = []

    for sen in corpus:
        tensor.append(s.EncodeAsIds(sen))

    with open("./korean_spm.vocab", 'r') as f:
        vocab = f.readlines()
 
    word_index = {}
    index_word = {}

    for idx, line in enumerate(vocab):
        word = line.split("\t")[0]

        word_index.update({word:idx})
        index_word.update({idx:word})

    #tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

    return tensor, word_index, index_word

In [43]:
tensor, word_index, index_word = sp_tokenize(s, corpus)

In [44]:
num_tokens = [len(tokens) for tokens in tensor]
num_tokens = np.array(num_tokens)

# 평균값, 최댓값, 표준편차
print(f"토큰 길이 평균: {np.mean(num_tokens)}")
print(f"토큰 길이 최대: {np.max(num_tokens)}")
print(f"토큰 길이 표준편차: {np.std(num_tokens)}")

max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
maxlen = int(max_tokens)
print(f'설정 최대 길이: {maxlen}')
print(f'전체 문장의 {np.sum(num_tokens < max_tokens) / len(num_tokens)}%가 설정값인 {maxlen}에 포함됩니다.')

In [45]:
# 전체 샘플 중 길이가 max_len 이하인 샘플의 비율이 몇 %인지 확인

def below_threshold_len(max_len, nested_list):
    cnt = 0
    for s in nested_list:
        if(len(s) <= max_len):
            cnt = cnt + 1
    print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (cnt / len(nested_list))*100))

In [46]:
max_len = 60
below_threshold_len(max_len, tensor)

In [47]:
X = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='pre', maxlen=60)

In [48]:
X.shape

In [49]:
train.label.shape

In [90]:
import pandas as pd
import sentencepiece as spm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM, Dropout, Conv1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

In [51]:
y = train['label']

In [52]:
y.shape

In [62]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=True, stratify=y, random_state=42)

# X_train = preprocess_data(X_train, sp)
# X_test = preprocess_data(X_test, sp)

# 모델 구성하기

- 모델 선정 이유
    >- 기존에 네이버 리뷰 프로젝트와 똑같이 모델을 사용하여 같은 환경에서 토큰에 따라 성능이 어떻게 변화할 수 있는지 확인하기 위해 같은 모델을 사용하였다.
    >- 시계열에서 좋은 성능을 가지는 lstm, stacked lstm, 1d cnn 모델을 테스트하였다. 

- Metrics 선정 이유
    >- 데이터 label 분포를 확인해본결과 거의 반반으로 분포해 있어서 acc를 사용하였다

- Loss 선정 이유
    >- 긍정, 부정을 분류하는 task이기에 binary_crossentropy 를 사용하였다.


### LSTM 모델

In [86]:
# 모델 설계
tf.keras.backend.clear_session()

model = Sequential()
model.add(Embedding(vocab_size, 100))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))

In [87]:
model.summary()

In [88]:
# 모델 검증

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_lstm_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [89]:
# 모델 훈련

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val), callbacks=[es, mc])

In [90]:
def draw_graph(history):
    history_dict = history.history
    try:
        acc = history_dict['accuracy']
        val_acc = history_dict['val_accuracy']
    except:
        acc = history_dict['acc']
        val_acc = history_dict['val_acc']
    loss = history_dict['loss']
    val_loss = history_dict['val_loss']

    epochs = range(1, len(acc) + 1)

    fig = plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(epochs, loss, 'r', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, acc, 'r', label='Training accuracy')
    plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
    plt.title('Training and validation accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.show()

In [91]:
history_dict = history.history
print(history_dict.keys())

In [92]:
draw_graph(history)

In [93]:
def predict_x(s, corpus):
    
    tensor = []
    for sen in corpus:
        tensor.append(s.EncodeAsIds(sen))

    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='pre', maxlen=42)

    return tensor

In [94]:
X_test = predict_x(s, test.document.tolist())
y_test = test.label

In [95]:
loaded_model = load_model('best_lstm_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

# stacked lstm

In [68]:
# 모델 학습
def create_model(vocab_size, word_vector_dim):
    model = Sequential([
        Embedding(vocab_size, word_vector_dim, input_shape=(None,)),
        LSTM(8, return_sequences=True),
        LSTM(16, return_sequences=False),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

vocab_size = len(s)  # SentencePiece의 vocab size
word_vector_dim = 300
stacked_lstm_model = create_model(vocab_size, word_vector_dim)
stacked_lstm_model.summary()

In [69]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
mc = ModelCheckpoint('stacked_lstm_best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

In [70]:
stacked_lstm_history = stacked_lstm_model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val), callbacks=[es, mc])

In [74]:
draw_graph(stacked_lstm_history)

In [84]:
loaded_model = load_model('stacked_lstm_best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

## 모델링 1D CNN

In [71]:
embedding_dim = 256 # 임베딩 벡터의 차원
dropout_ratio = 0.5 # 드롭아웃 비율
num_filters = 256 # 커널의 수
kernel_size = 3 # 커널의 크기
hidden_units = 128 # 뉴런의 수

cnn_model = Sequential()
cnn_model.add(Embedding(vocab_size, embedding_dim))
cnn_model.add(Dropout(dropout_ratio))
cnn_model.add(Conv1D(num_filters, kernel_size, padding='valid', activation='relu'))
cnn_model.add(GlobalMaxPooling1D())
cnn_model.add(Dense(hidden_units, activation='relu'))
cnn_model.add(Dropout(dropout_ratio))
cnn_model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
mc = ModelCheckpoint('cnn_best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
cnn_model.summary()

In [72]:
cnn_history = model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val), callbacks=[es, mc])

In [73]:
history_dict = history.history
print(history_dict.keys())

In [75]:
draw_graph(cnn_history)

In [85]:
loaded_model = load_model('cnn_best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

## Sentence Piece 결과 정리 (test acc 기준)
- lstm model : 0.8601
- stacked lstm model : 0.8517
- 1d cnn model : 0.8557

## 구현된 토크나이저를 적용하여 네이버 영화리뷰 감정 분석 모델을 재학습하기

# KoNLPy 형태소 분석기를 사용한 모델과 성능 비교하기

- 꼬꼬마(kkma)는 java 용량 문제 및 속도가 너무 느려서 수많은 시도 끝에 테스트하기 어렵다는 판단을 내렸다

In [9]:
hannanum = Hannanum()
kkma = Kkma(max_heap_size=2048)
komoran = Komoran()
mecab = Mecab()
okt = Okt()

In [101]:
tokenizer_list = [hannanum, kkma, komoran, mecab, okt]

# kor_text = '별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네'
kor_text = '노래도 너무즐겁고 내용도 아기자기 귀여워요동화같은ㅠ근데 아이들 너무귀엽다♥'

for tokenizer in tokenizer_list:
    print('[{}] \n{}'.format(tokenizer.__class__.__name__, tokenizer.pos(kor_text)))

In [ ]:
# kkma_doc = train.document.progress_apply(lambda x: kkma.pos(x)).tolist()

In [10]:
train.columns

In [43]:
tokenizer_list = [hannanum, komoran, mecab, okt]

for tokenizer in tokenizer_list:
    name = str(tokenizer.__class__.__name__)
    print(f'[{name}]')
    
    if name not in train.columns:
        train[f'{name}'] = train.document.progress_apply(lambda x: tokenizer.morphs(x))
    
        if name == 'Okt':
            train[f'{name}_stem'] = train.document.progress_apply(lambda x: tokenizer.morphs(x, stem=True))
        
        train.to_csv('data_preprocessed_morph_final.csv', index=False)
    else:
        pass

In [45]:
train.to_csv('data_preprocessed_morph_final.csv', index=False)

In [47]:
train.head()

## 형태소 분석기 별 성능 비교

In [52]:
test = preprocessing(test)

In [54]:
tokenizer_list = [hannanum, komoran, mecab, okt]

for tokenizer in tokenizer_list:
    name = str(tokenizer.__class__.__name__)
    print(f'[{name}]')
    
    if name not in test.columns:
        test[f'{name}'] = test.document.progress_apply(lambda x: tokenizer.morphs(x))
    
        if name == 'Okt':
            test[f'{name}_stem'] = test.document.progress_apply(lambda x: tokenizer.morphs(x, stem=True))
        
        test.to_csv('test_data_preprocessed_morph_final.csv', index=False)
    else:
        pass

In [57]:
tokenizer_list

In [60]:
train.head(2)

In [61]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [66]:
# 불용어 정의
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','을','으로','자',
             '에','와','한','하다','하','어','다','네','요','에서','에게','게','ㄴ','에서','고','로']

In [69]:
def compare_morphs(df, test, tokenizer, stopwords=stopwords):
    
    col = tokenizer.__class__.__name__

    X_data = df[col].apply(lambda x: [str(i) for i in x]).dropna().tolist()
    y_data = df['label'].dropna()

    print(len(X_data), len(y_data))
    
    # 형태소 분석
    #test_morphs = test.document.progress_apply(lambda x: tokenizer.morphs(x))
    test_morphs = test[col].copy()
    test_morphs = test_morphs.progress_apply(lambda x: [str(word) for word in x if not word in stopwords])

    # 토큰화
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(X_data)

    print(len(tokenizer.word_index))

    # 등장 빈도수가 3회 미만인 단어들의 분포 확인

    threshold = 3
    total_cnt = len(tokenizer.word_index) # 단어의 수
    rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
    total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
    rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

    # 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
    for key, value in tokenizer.word_counts.items():
        total_freq = total_freq + value

        # 단어의 등장 빈도수가 threshold보다 작으면
        if(value < threshold):
            rare_cnt = rare_cnt + 1
            rare_freq = rare_freq + value

    print('단어 집합(vocabulary)의 크기 :',total_cnt)
    print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
    print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
    print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

    # 등장 빈도수가 2이하인 단어들의 수를 제외한 단어의 개수를 단어 집합의 최대 크기로 제한
    # 전체 단어 개수 중 빈도수 2이하인 단어는 제거.
    # 0번 패딩 토큰을 고려하여 + 1

    vocab_size = total_cnt - rare_cnt + 1
    print('단어 집합의 크기 :',vocab_size)

    # 이를 케라스 토크나이저의 인자로 넘겨 텍스트 시퀀스를 숫자 시퀀스로 변환
    
    tokenizer = Tokenizer(vocab_size) 
    tokenizer.fit_on_texts(X_data)
    X_train = tokenizer.texts_to_sequences(X_data)
    X_test = tokenizer.texts_to_sequences(test_morphs)
    
    return X_train, X_test, tokenizer, vocab_size

In [71]:
X_train, X_test, tokenizer, vocab_size = compare_morphs(train, test, okt)

In [73]:
y_train = y_data.tolist()

In [74]:
len(X_train), len(y_train)

In [75]:
def drop_empty_list(X_train, y_train):
    # 각 샘플들의 길이를 확인해서 길이가 0인 샘플들의 인덱스 받아오기
    drop_train = [index for index, sentence in enumerate(X_train) if len(sentence) < 1]
    print(f'빈 샘플 수 : {len(drop_train)}')

    # 빈 샘플 제거
    X_train = np.delete(X_train, drop_train, axis=0)
    y_train = np.delete(y_train, drop_train, axis=0)
    print(f'빈 샘플 제거 후 남은 X train data : {len(X_train)}')
    print(f'빈 샘플 제거 후 남은 y train data : {len(y_train)}')
    return X_train, y_train

In [76]:
X_train, y_train = drop_empty_list(X_train, y_train)

In [77]:
print('리뷰의 최대 길이 :',max(len(l) for l in X_train))
print('리뷰의 평균 길이 :',sum(map(len, X_train))/len(X_train))
plt.hist([len(s) for s in X_train], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show();

In [78]:
# 전체 샘플 중 길이가 max_len 이하인 샘플의 비율이 몇 %인지 확인

def below_threshold_len(max_len, nested_list):
    cnt = 0
    for s in nested_list:
        if(len(s) <= max_len):
            cnt = cnt + 1
    print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (cnt / len(nested_list))*100))

In [80]:
max_len = 50
below_threshold_len(max_len, X_train)

In [97]:
def padding_and_split(X_train, X_test, y_train, max_len):
    X_train = pad_sequences(X_train, maxlen = max_len)
    X_test = pad_sequences(X_test, maxlen = max_len)

    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, shuffle=True,
                                                      stratify=y_train, random_state=777)
    return X_train, X_val, y_train, y_val, X_test

In [98]:
X_train, X_val, y_train, y_val, X_test = padding_and_split(X_train, X_test, y_train, max_len)

In [86]:
vocab_size

In [87]:
embedding_dim = 100

In [88]:
def lstm_model(vocab_size, embedding_dim, optimizer='rmsprop'):
    model = Sequential([
        Embedding(vocab_size, embedding_dim),
        LSTM(128),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['acc'])
    return model

In [91]:
model = lstm_model(vocab_size, embedding_dim)

In [92]:
# 모델 검증

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_lstm_model_okt.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [93]:
# 모델 훈련
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val), callbacks=[es, mc])

In [120]:
def draw_graph(history):
    history_dict = history.history
    try:
        acc = history_dict['accuracy']
        val_acc = history_dict['val_accuracy']
    except:
        acc = history_dict['acc']
        val_acc = history_dict['val_acc']
    loss = history_dict['loss']
    val_loss = history_dict['val_loss']

    epochs = range(1, len(acc) + 1)

    fig = plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(epochs, loss, 'r', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, acc, 'r', label='Training accuracy')
    plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
    plt.title('Training and validation accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.show()

In [ ]:
draw_graph(history)

In [100]:
X_test.shape

In [105]:
y_test = np.array(test.label)

In [106]:
# 테스트 정확도 측정

loaded_model = load_model('best_lstm_model_okt.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

## 한나눔 성능

In [108]:
X_train, X_test, tokenizer, vocab_size = compare_morphs(train, test, hannanum)

In [110]:
y_train = y_data.tolist()

In [111]:
X_train, y_train = drop_empty_list(X_train, y_train)

In [112]:
print('리뷰의 최대 길이 :',max(len(l) for l in X_train))
print('리뷰의 평균 길이 :',sum(map(len, X_train))/len(X_train))
plt.hist([len(s) for s in X_train], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show();

In [113]:
max_len = 50
below_threshold_len(max_len, X_train)

In [114]:
X_train, X_val, y_train, y_val, X_test = padding_and_split(X_train, X_test, y_train, max_len)

In [116]:
embedding_dim = 100

In [117]:
model = lstm_model(vocab_size, embedding_dim)

In [118]:
# 모델 검증

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_lstm_model_hannanum.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [119]:
# 모델 훈련
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val), callbacks=[es, mc])

In [121]:
draw_graph(history)

In [122]:
X_test.shape

In [123]:
y_test = np.array(test.label)

In [125]:
y_test.shape

In [126]:
# 테스트 정확도 측정

loaded_model = load_model('best_lstm_model_hannanum.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

## Komoran 성능

In [148]:
name = komoran.__class__.__name__.lower()

In [137]:
X_train, X_test, tokenizer, vocab_size = compare_morphs(train, test, komoran)

In [138]:
y_train = y_data.tolist()

In [139]:
X_train, y_train = drop_empty_list(X_train, y_train)

In [140]:
print('리뷰의 최대 길이 :',max(len(l) for l in X_train))
print('리뷰의 평균 길이 :',sum(map(len, X_train))/len(X_train))
plt.hist([len(s) for s in X_train], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show();

In [142]:
max_len = 70
below_threshold_len(max_len, X_train)

In [143]:
X_train, X_val, y_train, y_val, X_test = padding_and_split(X_train, X_test, y_train, max_len)

In [144]:
embedding_dim = 100

In [145]:
model = lstm_model(vocab_size, embedding_dim)

In [149]:
# 모델 검증

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint(f'best_lstm_model_{name}.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [150]:
# 모델 훈련
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val), callbacks=[es, mc])

In [151]:
draw_graph(history)

In [152]:
X_test.shape

In [153]:
y_test = np.array(test.label)

In [125]:
y_test.shape

In [154]:
# 테스트 정확도 측정

loaded_model = load_model(f'best_lstm_model_{name}.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

# Mecab 성능

In [155]:
name = mecab.__class__.__name__.lower()

X_train, X_test, tokenizer, vocab_size = compare_morphs(train, test, mecab)
y_train = y_data.tolist()
X_train, y_train = drop_empty_list(X_train, y_train)

In [156]:
print('리뷰의 최대 길이 :',max(len(l) for l in X_train))
print('리뷰의 평균 길이 :',sum(map(len, X_train))/len(X_train))
plt.hist([len(s) for s in X_train], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show();

In [157]:
max_len = 70
below_threshold_len(max_len, X_train)

In [158]:
X_train, X_val, y_train, y_val, X_test = padding_and_split(X_train, X_test, y_train, max_len)

In [159]:
embedding_dim = 100

model = lstm_model(vocab_size, embedding_dim)

# 모델 검증
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint(f'best_lstm_model_{name}.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

# 모델 훈련
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val), callbacks=[es, mc])

In [160]:
draw_graph(history)

In [161]:
X_test.shape

In [162]:
y_test = np.array(test.label)

In [163]:
y_test.shape

In [164]:
# 테스트 정확도 측정

loaded_model = load_model(f'best_lstm_model_{name}.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

## Okt (stem=True) 성능

In [167]:
df = train.copy()

In [168]:
col = 'Okt_stem'
name = col.lower()

X_data = df[col].apply(lambda x: [str(i) for i in x]).dropna().tolist()
y_data = df['label'].dropna()

print(len(X_data), len(y_data))

# 형태소 분석
#test_morphs = test.document.progress_apply(lambda x: tokenizer.morphs(x))
test_morphs = test[col].copy()
test_morphs = test_morphs.progress_apply(lambda x: [str(word) for word in x if not word in stopwords])

# 토큰화
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_data)

print(len(tokenizer.word_index))

# 등장 빈도수가 3회 미만인 단어들의 분포 확인

threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

# 등장 빈도수가 2이하인 단어들의 수를 제외한 단어의 개수를 단어 집합의 최대 크기로 제한
# 전체 단어 개수 중 빈도수 2이하인 단어는 제거.
# 0번 패딩 토큰을 고려하여 + 1

vocab_size = total_cnt - rare_cnt + 1
print('단어 집합의 크기 :',vocab_size)

# 이를 케라스 토크나이저의 인자로 넘겨 텍스트 시퀀스를 숫자 시퀀스로 변환

tokenizer = Tokenizer(vocab_size) 
tokenizer.fit_on_texts(X_data)
X_train = tokenizer.texts_to_sequences(X_data)
X_test = tokenizer.texts_to_sequences(test_morphs)

In [169]:
y_train = y_data.tolist()
X_train, y_train = drop_empty_list(X_train, y_train)

In [170]:
print('리뷰의 최대 길이 :',max(len(l) for l in X_train))
print('리뷰의 평균 길이 :',sum(map(len, X_train))/len(X_train))
plt.hist([len(s) for s in X_train], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show();

In [171]:
max_len = 70
below_threshold_len(max_len, X_train)

In [172]:
X_train, X_val, y_train, y_val, X_test = padding_and_split(X_train, X_test, y_train, max_len)

In [173]:
embedding_dim = 100

model = lstm_model(vocab_size, embedding_dim)

# 모델 검증
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint(f'best_lstm_model_{name}.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

# 모델 훈련
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val), callbacks=[es, mc])

In [174]:
draw_graph(history)

In [175]:
X_test.shape

In [176]:
y_test = np.array(test.label)

In [177]:
y_test.shape

In [178]:
# 테스트 정확도 측정

loaded_model = load_model(f'best_lstm_model_{name}.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

## Sentence Piece 결과 정리 (test acc 기준)
- lstm model : 0.8601
- stacked lstm model : 0.8517
- 1d cnn model : 0.8557

## 최종 형태소별 성능 정리 ( lstm model )

- hannanum : 0.8199 (빈 샘플 수 : 5513)
- komoran : 0.8544 (빈 샘플 수 : 2278)
- mecab : 0.8649 (빈 샘플 수 : 202)
- okt
    - stem = False : 0.8586 (빈 샘플 수 : 395)
    - stem = True : 0.8613 (빈 샘플 수 : 241)


- 빈 샘플 수에 비례해 성능이 좋아지는 것을 볼 수 있다
- 이에 따라 성능 향상에 토큰을 잘 나누는 것이 매우 중요한 요소로 보인다

## SentencePiece 모델의 model_type, vocab_size 등을 변경해 가면서 성능 개선 여부 확인하기

- mecab의 vocab size가 21741이었으므로 이를 따라 해본다

In [180]:
temp_file = os.getenv('HOME')+'/aiffel/sp_tokenizer/data/korean-english-nsmc.train.ko.temp'

vocab_size = 21741

with open(temp_file, 'w') as f:
    for row in corpus:   # 이전에 나왔던 정제했던 corpus를 활용해서 진행해야 합니다.
        f.write(str(row) + '\n')

spm.SentencePieceTrainer.Train(
    '--input={} --model_prefix=korean_spm --vocab_size={}'.format(temp_file, vocab_size)    
)

In [186]:
s = spm.SentencePieceProcessor()
s.Load('korean_spm.model')

In [187]:
test_data = test.document.tolist()

In [188]:
def sp_tokenize(s, corpus): 

    tensor = []

    for sen in corpus:
        tensor.append(s.EncodeAsIds(sen))

    with open("./korean_spm.vocab", 'r') as f:
        vocab = f.readlines()
 
    word_index = {}
    index_word = {}

    for idx, line in enumerate(vocab):
        word = line.split("\t")[0]

        word_index.update({word:idx})
        index_word.update({idx:word})

    #tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

    return tensor, word_index, index_word

In [189]:
tensor, word_index, index_word = sp_tokenize(s, corpus)

In [190]:
num_tokens = [len(tokens) for tokens in tensor]
num_tokens = np.array(num_tokens)

# 평균값, 최댓값, 표준편차
print(f"토큰 길이 평균: {np.mean(num_tokens)}")
print(f"토큰 길이 최대: {np.max(num_tokens)}")
print(f"토큰 길이 표준편차: {np.std(num_tokens)}")

max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
maxlen = int(max_tokens)
print(f'설정 최대 길이: {maxlen}')
print(f'전체 문장의 {np.sum(num_tokens < max_tokens) / len(num_tokens)}%가 설정값인 {maxlen}에 포함됩니다.')

In [191]:
max_len = 60
below_threshold_len(max_len, tensor)

In [192]:
X = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='pre', maxlen=60)

In [193]:
X.shape

In [194]:
train.label.shape

In [195]:
y = train['label']

In [196]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=True, stratify=y, random_state=42)

In [198]:
name = 'sentencepiece_10000'

In [199]:
embedding_dim = 100

model = lstm_model(vocab_size, embedding_dim)

# 모델 검증
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint(f'best_lstm_model_{name}.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

# 모델 훈련
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val), callbacks=[es, mc])

In [200]:
draw_graph(history)

In [201]:
def predict_x(s, corpus):
    
    tensor = []
    for sen in corpus:
        tensor.append(s.EncodeAsIds(sen))

    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='pre', maxlen=42)

    return tensor

In [202]:
X_test = predict_x(s, test.document.tolist())
y_test = test.label

In [203]:
loaded_model = load_model('best_lstm_model_sentencepiece_10000.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

## bpe

In [207]:
import sentencepiece as spm
import os

temp_file = os.getenv('HOME')+'/aiffel/sp_tokenizer/data/korean-english-nsmc.train.ko.temp'

vocab_size = 8000

with open(temp_file, 'w') as f:
    for row in corpus:   # 이전에 나왔던 정제했던 corpus를 활용해서 진행해야 합니다.
        f.write(str(row) + '\n')

spm.SentencePieceTrainer.Train(
    '--input={} --model_prefix=korean_spm --vocab_size={} --model_type=bpe'.format(temp_file, vocab_size)    
)

In [208]:
s = spm.SentencePieceProcessor()
s.Load('korean_spm.model')

In [209]:
test_data = test.document.tolist()

In [210]:
def sp_tokenize(s, corpus): 

    tensor = []

    for sen in corpus:
        tensor.append(s.EncodeAsIds(sen))

    with open("./korean_spm.vocab", 'r') as f:
        vocab = f.readlines()
 
    word_index = {}
    index_word = {}

    for idx, line in enumerate(vocab):
        word = line.split("\t")[0]

        word_index.update({word:idx})
        index_word.update({idx:word})

    #tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

    return tensor, word_index, index_word

In [211]:
tensor, word_index, index_word = sp_tokenize(s, corpus)

In [212]:
num_tokens = [len(tokens) for tokens in tensor]
num_tokens = np.array(num_tokens)

# 평균값, 최댓값, 표준편차
print(f"토큰 길이 평균: {np.mean(num_tokens)}")
print(f"토큰 길이 최대: {np.max(num_tokens)}")
print(f"토큰 길이 표준편차: {np.std(num_tokens)}")

max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
maxlen = int(max_tokens)
print(f'설정 최대 길이: {maxlen}')
print(f'전체 문장의 {np.sum(num_tokens < max_tokens) / len(num_tokens)}%가 설정값인 {maxlen}에 포함됩니다.')

In [213]:
max_len = 60
below_threshold_len(max_len, tensor)

In [214]:
X = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='pre', maxlen=60)

In [215]:
X.shape

In [216]:
train.label.shape

In [217]:
y = train['label']

In [218]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=True, stratify=y, random_state=42)

In [219]:
name = 'sentencepiece_bpe'

In [220]:
embedding_dim = 100

model = lstm_model(vocab_size, embedding_dim)

# 모델 검증
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint(f'best_lstm_model_{name}.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

# 모델 훈련
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val), callbacks=[es, mc])

In [221]:
draw_graph(history)

In [222]:
X_test = predict_x(s, test.document.tolist())
y_test = test.label

In [223]:
loaded_model = load_model('best_lstm_model_sentencepiece_bpe.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

In [243]:
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import roc_auc_score

In [225]:
pred = loaded_model.predict(X_test)

In [233]:
y_pred = np.round(pred).flatten().tolist()

In [230]:
y_true = y_test.tolist()

In [236]:
print(classification_report(y_true, y_pred))

In [239]:
confusion_matrix(y_true, y_pred)

In [244]:
roc_auc_score(y_true, pred)

In [248]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from tensorflow.keras.models import load_model

def get_roc_curve(model_path, X_test, y_test):
    # 모델 불러오기
    loaded_model = load_model(model_path)

    # 테스트 데이터에 대한 예측 수행
    y_pred_prob = loaded_model.predict(X_test)  # 모델이 예측한 확률 값

    # ROC 곡선 계산
    fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
    roc_auc = auc(fpr, tpr)

    # ROC 곡선 그리기
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc="lower right")
    plt.show()

    print("\n테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))
    print("ROC AUC: %.4f" % roc_auc)

In [249]:
path = 'best_lstm_model_sentencepiece_bpe.h5'
get_roc_curve(path, X_test, y_test)

# 최종 성능 비교 및 결론

## Sentence Piece 결과 정리 (test acc 기준)
- lstm model : 0.8601
- stacked lstm model : 0.8517
- 1d cnn model : 0.8557

## 최종 형태소별 성능 정리 ( lstm model )

- hannanum : 0.8199 (빈 샘플 수 : 5513)
- komoran : 0.8544 (빈 샘플 수 : 2278)
- mecab : 0.8649 (빈 샘플 수 : 202)
- okt
    - stem = False : 0.8586 (빈 샘플 수 : 395)
    - stem = True : 0.8613 (빈 샘플 수 : 241)


- 빈 샘플 수에 비례해 성능이 좋아지는 것을 볼 수 있다
- 이에 따라 성능 향상에 토큰을 잘 나누는 것이 매우 중요한 요소로 보인다

## SentencePiece vocab size, model type 변화
- vocab_size = 21741로 늘림 : 0.8608
- model type = bpe로 변경 : 0.8601

    - 성능 향상에 큰 변화가 없었다

# 회고
- 배운 점 
    - 언젠가 한번 쯤은 형태소 분석기별로 성능을 비교해보고 싶었는데 이번 기회로 제대로 비교할 수 있었던 좋은 기회였다
    - 좀 더 task specific하게 전처리를 진행해볼 수 있어 좋은 기회였다
- 아쉬운 점 
    - 좀 더 체계적으로 비교를 해보고 싶었는데 시간상의 이유로 제대로 비교해보지 못해서 아쉽다
- 느낀 점
    - task 별로 성능이 좋은 모델, 토큰화 방식이 다를 수 있다.
- 어려웠던 점 
    - 코드를 좀 더 재사용성있게 짜는 방법을 더 고민해봐야겠다